In [11]:
from sklearn.naive_bayes import MultinomialNB

import os
os.chdir('/Users/kshitijphulare/DLH-Proj/')
from pre_processing import *
from main import *
# from dataset.preprocessing.tf_idf_all_feature_matrix_gen import TFIDFFeatureGeneration
from sklearn.metrics import f1_score
import numpy as np
from sklearn.model_selection import KFold
import csv
import pandas as pd

In [12]:
class NaiveBayes:
    def __init__(self, x_train, y_train, x_test, y_test):
        self.nb = MultinomialNB()
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        # print(self.x_train.shape, self.y_train.shape, self.x_test.shape, self.y_test.shape)
        
    def train(self):
        self.nb.fit(self.x_train, self.y_train)

    def test_and_evaluate(self):
        y_pred = self.nb.predict(self.x_test)
        f1_macro = f1_score(self.y_test, y_pred, average='macro')
        f1_micro = f1_score(self.y_test, y_pred, average='micro')
        #print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")
        return f1_macro, f1_micro

In [18]:
morbidities = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'Obesity', 'OSA', 'PVD', 'Venous_Insufficiency']

In [19]:
column_headings = ["Morbidity Class", "NB_Macro F1", "NB_Micro F1"]

with open("./performance_NB.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    

In [20]:
total_macro=[]
total_micro = []

In [21]:
for morbidity in morbidities:
    train_preprocessed_df = pd.read_csv('./dataset/train/train_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = tf_idf(train_preprocessed_df, morbidity)
    # add KFold cross validation
    skf = KFold(n_splits=10, shuffle=True, random_state=42)
    
    f1_macro_list = []
    f1_micro_list = []
    for train_idx, val_idx in skf.split(X, Y):
        X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
        X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

        # Training RF using TF-IDF Representation
        nb_obj = NaiveBayes(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold)
        nb_obj.train()

        f1_macro, f1_micro = nb_obj.test_and_evaluate()

        f1_macro_list.append(f1_macro)
        f1_micro_list.append(f1_micro)

    f1_macro = np.mean(f1_macro_list)
    f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    total_macro.append(f1_macro)
    total_micro.append(f1_micro)
    with open("./performance_NB.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)
    
with open("./performance_NB.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ['Average']
    row.extend([sum(total_macro)/len(total_macro),sum(total_micro)/len(total_micro)])
    writer.writerow(row)


(572, 600) (572,) Counter({0.0: 502, 1.0: 70})
Macro F1 score: 0.4672986632039369 and Micro F1 Score 0.8776164549304294
(548, 600) (548,) Counter({1.0: 325, 0.0: 223})
Macro F1 score: 0.7095810085794834 and Micro F1 Score 0.7464646464646465
(243, 600) (243,) Counter({1.0: 243})
Macro F1 score: 1.0 and Micro F1 Score 1.0
(582, 600) (582,) Counter({0.0: 460, 1.0: 122})
Macro F1 score: 0.46329019107041497 and Micro F1 Score 0.793834015195792
(567, 600) (567,) Counter({1.0: 396, 0.0: 171})
Macro F1 score: 0.4217886390468421 and Micro F1 Score 0.6985902255639098
(593, 600) (593,) Counter({0.0: 506, 1.0: 87})
Macro F1 score: 0.4599902140879871 and Micro F1 Score 0.8532485875706215
(487, 600) (487,) Counter({0.0: 372, 1.0: 115})
Macro F1 score: 0.4323776782842145 and Micro F1 Score 0.7638605442176871
(596, 600) (596,) Counter({0.0: 518, 1.0: 78})
Macro F1 score: 0.4647957966912661 and Micro F1 Score 0.8691242937853108
(502, 600) (502,) Counter({1.0: 262, 0.0: 240})
Macro F1 score: 0.607527535